In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [2]:
conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "~/bin/spark-3.0.3-bin-hadoop3.2/jars/gcs-connector-hadoop3-latest.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "~/.google/credentials/google_credentials.json")

In [3]:
sc = SparkContext(conf=conf)

22/04/02 14:01:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
22/04/02 14:01:47 WARN DependencyUtils: Local jar /home/gerry/uk_crime_data/notebooks/~/bin/spark-3.0.3-bin-hadoop3.2/jars/gcs-connector-hadoop3-latest.jar does not exist, skipping.
22/04/02 14:01:47 INFO SparkContext: Running Spark version 3.0.3
22/04/02 14:01:47 INFO ResourceUtils: ==============================================================
22/04/02 14:01:47 INFO ResourceUtils: Resources for spark.driver:

22/04/02 14:01:47 INFO ResourceUtils: ==============================================================
22/04/02 14:01:47 INFO SparkContext: Submitted application: test
22/04/02 14:01:47 INFO SecurityManager: Changing view acls to: gerry
22/04/02 14:01:47 INFO SecurityManager: Changing modify acls to: gerry
22/04/02 14:01:47 INFO SecurityManager: Changing view ac

In [4]:
sc._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
sc._jsc.hadoopConfiguration().set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
sc._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.json.keyfile", "/home/gerry/.google/credentials/google_credentials.json")
sc._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.enable", "true")


In [5]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()


22/04/02 14:02:02 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir ('file:/home/gerry/uk_crime_data/notebooks/spark-warehouse').
22/04/02 14:02:02 INFO SharedState: Warehouse path is 'file:/home/gerry/uk_crime_data/notebooks/spark-warehouse'.


In [6]:
bucket_name = "dtc_data_lake_de-bootcamp-339509"
path = f"gs://{bucket_name}/metropolitan_2020-01.parquet"
path

'gs://dtc_data_lake_de-bootcamp-339509/metropolitan_2020-01.parquet'

In [7]:
df  = spark.read.parquet(path)

22/04/02 14:02:09 INFO InMemoryFileIndex: It took 146 ms to list leaf files for 1 paths.
22/04/02 14:02:10 INFO SparkContext: Starting job: parquet at NativeMethodAccessorImpl.java:0
22/04/02 14:02:10 INFO DAGScheduler: Got job 0 (parquet at NativeMethodAccessorImpl.java:0) with 1 output partitions
22/04/02 14:02:10 INFO DAGScheduler: Final stage: ResultStage 0 (parquet at NativeMethodAccessorImpl.java:0)
22/04/02 14:02:10 INFO DAGScheduler: Parents of final stage: List()
22/04/02 14:02:10 INFO DAGScheduler: Missing parents: List()
22/04/02 14:02:10 INFO DAGScheduler: Submitting ResultStage 0 (MapPartitionsRDD[1] at parquet at NativeMethodAccessorImpl.java:0), which has no missing parents
22/04/02 14:02:10 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 84.9 KiB, free 434.3 MiB)
22/04/02 14:02:10 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 30.3 KiB, free 434.3 MiB)
22/04/02 14:02:10 INFO BlockManagerInfo: Added bro

In [22]:
df.show()

22/04/02 13:45:40 INFO FileSourceStrategy: Pushed Filters: 
22/04/02 13:45:40 INFO FileSourceStrategy: Post-Scan Filters: 
22/04/02 13:45:40 INFO FileSourceStrategy: Output Data Schema: struct<age_range: string, outcome: string, involved_person: boolean, self_defined_ethnicity: string, gender: string ... 19 more fields>
22/04/02 13:45:41 INFO CodeGenerator: Code generated in 401.677735 ms
22/04/02 13:45:41 INFO MemoryStore: Block broadcast_1 stored as values in memory (estimated size 180.8 KiB, free 434.2 MiB)
22/04/02 13:45:41 INFO MemoryStore: Block broadcast_1_piece0 stored as bytes in memory (estimated size 29.3 KiB, free 434.2 MiB)
22/04/02 13:45:41 INFO BlockManagerInfo: Added broadcast_1_piece0 in memory on de-bootcamp.europe-west1-b.c.de-bootcamp-339509.internal:39923 (size: 29.3 KiB, free: 434.4 MiB)
22/04/02 13:45:41 INFO SparkContext: Created broadcast 1 from showString at NativeMethodAccessorImpl.java:0
22/04/02 13:45:41 INFO FileSourceScanExec: Planning scan with bin packi

+---------+--------------------+---------------+----------------------+------+--------------------+----------------------------------+-------------------+-----------------------------------+--------------------+--------------------+---------+-------------------------+--------------------+--------------+--------------------+---------+---------+-------------------+---------+--------------------+
|age_range|             outcome|involved_person|self_defined_ethnicity|gender|         legislation|outcome_linked_to_object_of_search|           datetime|removal_of_more_than_outer_clothing|      outcome_object|            location|operation|officer_defined_ethnicity|                type|operation_name|    object_of_search| latitude|longitude|           lat_long|street_id|         street_name|
+---------+--------------------+---------------+----------------------+------+--------------------+----------------------------------+-------------------+-----------------------------------+----------------

22/04/02 13:45:42 INFO Executor: Finished task 0.0 in stage 1.0 (TID 1). 4299 bytes result sent to driver
22/04/02 13:45:43 INFO TaskSetManager: Finished task 0.0 in stage 1.0 (TID 1) in 1431 ms on de-bootcamp.europe-west1-b.c.de-bootcamp-339509.internal (executor driver) (1/1)
22/04/02 13:45:43 INFO TaskSchedulerImpl: Removed TaskSet 1.0, whose tasks have all completed, from pool 
22/04/02 13:45:43 INFO DAGScheduler: ResultStage 1 (showString at NativeMethodAccessorImpl.java:0) finished in 1.475 s
22/04/02 13:45:43 INFO DAGScheduler: Job 1 is finished. Cancelling potential speculative or zombie tasks for this job
22/04/02 13:45:43 INFO TaskSchedulerImpl: Killing all running tasks in stage 1: Stage finished
22/04/02 13:45:43 INFO DAGScheduler: Job 1 finished: showString at NativeMethodAccessorImpl.java:0, took 1.483682 s
22/04/02 13:45:43 INFO CodeGenerator: Code generated in 54.479083 ms


In [8]:
df.printSchema()

root
 |-- age_range: string (nullable = true)
 |-- outcome: string (nullable = true)
 |-- involved_person: boolean (nullable = true)
 |-- self_defined_ethnicity: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- legislation: string (nullable = true)
 |-- outcome_linked_to_object_of_search: string (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- removal_of_more_than_outer_clothing: string (nullable = true)
 |-- outcome_object: string (nullable = true)
 |-- location: string (nullable = true)
 |-- operation: string (nullable = true)
 |-- officer_defined_ethnicity: string (nullable = true)
 |-- type: string (nullable = true)
 |-- operation_name: string (nullable = true)
 |-- object_of_search: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- lat_long: string (nullable = true)
 |-- street_id: long (nullable = true)
 |-- street_name: string (nullable = true)



In [29]:
df = df.write.parquet('new_data2/')

22/04/02 13:54:29 INFO FileSourceStrategy: Pushed Filters: 
22/04/02 13:54:29 INFO FileSourceStrategy: Post-Scan Filters: 
22/04/02 13:54:29 INFO FileSourceStrategy: Output Data Schema: struct<age_range: string, outcome: string, involved_person: boolean, self_defined_ethnicity: string, gender: string ... 19 more fields>
22/04/02 13:54:29 INFO ParquetFileFormat: Using default output committer for Parquet: org.apache.parquet.hadoop.ParquetOutputCommitter
22/04/02 13:54:29 INFO FileOutputCommitter: File Output Committer Algorithm version is 1
22/04/02 13:54:29 INFO FileOutputCommitter: FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
22/04/02 13:54:29 INFO SQLHadoopMapReduceCommitProtocol: Using user defined output committer class org.apache.parquet.hadoop.ParquetOutputCommitter
22/04/02 13:54:29 INFO FileOutputCommitter: File Output Committer Algorithm version is 1
22/04/02 13:54:29 INFO FileOutputCommitter: FileOutputCommit

22/04/02 13:54:29 INFO FileScanRDD: Reading File path: gs://dtc_data_lake_de-bootcamp-339509/metropolitan_2020-01.parquet, range: 0-1025904, partition values: [empty row]
22/04/02 13:54:31 INFO BlockManagerInfo: Removed broadcast_2_piece0 on de-bootcamp.europe-west1-b.c.de-bootcamp-339509.internal:39923 in memory (size: 8.2 KiB, free: 434.3 MiB)
22/04/02 13:54:31 INFO BlockManagerInfo: Removed broadcast_1_piece0 on de-bootcamp.europe-west1-b.c.de-bootcamp-339509.internal:39923 in memory (size: 29.3 KiB, free: 434.3 MiB)
22/04/02 13:54:31 INFO InternalParquetRecordWriter: Flushing mem columnStore to file. allocated memory: 3174311
22/04/02 13:54:31 INFO FileOutputCommitter: Saved output of task 'attempt_20220402135429444884042504427888_0002_m_000000_2' to file:/home/gerry/uk_crime_data/notebooks/new_data2/_temporary/0/task_20220402135429444884042504427888_0002_m_000000
22/04/02 13:54:31 INFO SparkHadoopMapRedUtil: attempt_20220402135429444884042504427888_0002_m_000000_2: Committed
22/04